In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from Helper_functions import *
import pandas as pd

In [ ]:
options = webdriver.ChromeOptions()
adblocker_path = "./adblock.crx"
options.add_extension(adblocker_path)
options.binary_location = './chrome-win64/chrome.exe'
driver = webdriver.Chrome(options=options)

driver.get("https://www.ratemyprofessors.com/search/professors/1490?q=*")
wait_for_element(driver, '//div[@id="ad-header-leaderboard"]')
actions = ActionChains(driver)

df = pd.DataFrame(columns = ["Reviews", "Score", "Sentiment"])

while True:
    more_profs = driver.find_elements(By.XPATH, "//button[contains(text(), 'Show More')]")
    if more_profs:
        driver.execute_script("arguments[0].click();", more_profs[0])
        time.sleep(1)
    else:
        break

profs = driver.find_elements(By.XPATH, "//a[contains(@class, 'TeacherCard')]")
for prof in profs:
    try:
        sentiment = []
        link = prof.get_attribute('href')
        driver.execute_script(f'window.open("{link}", "_blank");')
        driver.switch_to.window(driver.window_handles[1])
        wait_for_element(driver, "//canvas[@role='img']")
        while True:
            more_ratings = driver.find_elements(By.XPATH, "//button[contains(text(), 'Load More Ratings')]")
            if more_ratings:
                driver.execute_script("arguments[0].click();", more_ratings[0])
                time.sleep(1)
            else:
                break
        # print("hi")
        reviews_raw = driver.find_elements(By.XPATH, '//div[contains(@class, "Comments__StyledComments")]')
        if reviews_raw:
            reviews = [review.text for review in reviews_raw]
            scores = [score.text for score in driver.find_elements(By.XPATH, '//div[contains(text(), "Quality")]/following-sibling::*[1][self::div[contains(@class, "CardNumRating__CardNumRatingNumber-sc-17t4b9u-2")]]')]
        else:
            reviews = ["no reviews"]
            scores = ["no scores"]
        for score in scores:
            if score != "no scores":
                if 3.5 < float(score) <= 5:
                    sentiment.append("positive")
                elif 2.5 < float(score) <= 3.5:
                    sentiment.append("neutral")
                else:
                    sentiment.append("negative")
        # print(str(len(reviews)) + ", " + str(len(scores)) + ", " + str(len(sentiment)))
        df2 = pd.DataFrame(data={"Reviews": reviews, "Score":scores, "Sentiment":sentiment})
        df = pd.concat([df, df2], ignore_index = True)
        df.to_csv("./training_data.csv", index=False)
        driver.close()
        driver.switch_to.window(driver.window_handles[0])
        time.sleep(3)
    except Exception as e:
        print("Exception caught: " + str(e))
            
                
            
                    